# 训练胜负模型

In [588]:
import pandas as pd 
import zipfile
import os 
import py7zr
import multiprocessing 
from tqdm import tqdm 
import numpy as np
import pandas as pd
import shap
import joblib
import matplotlib.pyplot as plt
import multiprocessing
from tqdm import tqdm
import os
from concurrent.futures import ThreadPoolExecutor 
import numpy as np
import pandas as pd
import lightgbm
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, mean_squared_error, classification_report
from lightgbm import log_evaluation, early_stopping
from sklearn.preprocessing import KBinsDiscretizer
import random
plt.rcParams['axes.unicode_minus']=False
#6和3；7和3；8和3
DRtg_bins = 8
PM_bins = 3
DRtg_Fuzzification = True
# DRtg_Fuzzification = False
PM_Fuzzification = True
# PM_Fuzzification = False

def data_load(data):
    df = data
    # print(df.info())
    # print(df.columns)
    # 将时间转换为浮点数
    time_features = []
    for i in range(36):
        feature = 'MP' + '_' + str(i)
        time_features.append(feature)
    # print(time_features) 
    
    df = df[~df['BPM_0'].astype(str).str.contains(':')]
    # print(df['BPM_0'].astype(str).str.contains(':'))
    print('清洗后数据：',len(df))
    df[time_features] = df[time_features].fillna('0:0')
    df[time_features] = df[time_features].map(lambda x: '0:0' if ':' not in str(x) else x)
    df[time_features] =df[time_features].map(lambda x: float(x.replace(':', '.')))
    df = df.fillna(0)
    if  DRtg_Fuzzification:
        #  #缩小'ORtg','DRtg'影响
        norm_features = []
        for i in range(36):
            for feat in ['DRtg']:
                feature = feat + '_' + str(i)
                norm_features.append(feature)
        # df = df.fillna(0)
        # # 转换为数值类型
        df[norm_features] = df[norm_features].apply(pd.to_numeric, errors='coerce')
        # 将负值变为0
        df[norm_features] = df[norm_features].applymap(lambda x: 0 if x < 0 else x)
        # 将 norm_features 值聚合在一起
        norm_feature_values = df[norm_features].values.flatten().reshape(-1, 1)    
        print(norm_feature_values.shape)
        # 将数据划分为 10 个区间进行数值离散化处理
        kdb = KBinsDiscretizer(n_bins = DRtg_bins, encode = 'ordinal', strategy='quantile')
        kdb.fit(norm_feature_values)
        for norm_feature in norm_features:
            df[norm_feature] = kdb.transform(df[norm_feature].values.reshape(-1, 1))
    if  PM_Fuzzification:
        norm_features = []
        for i in range(36):
            for feat in ['+/-']:
                feature = feat + '_' + str(i)
                norm_features.append(feature)
        # # 转换为数值类型
        df[norm_features] = df[norm_features].apply(pd.to_numeric, errors='coerce')
        # 将负值变为0
        df[norm_features] = df[norm_features].applymap(lambda x: 0 if x < 0 else x)
        # 将 norm_features 值聚合在一起
        norm_feature_values = df[norm_features].values.flatten().reshape(-1, 1)    
        print(norm_feature_values.shape)
        # 将数据划分为 10 个区间进行数值离散化处理
        kdb = KBinsDiscretizer(n_bins = PM_bins, encode = 'ordinal', strategy='quantile')
        kdb.fit(norm_feature_values)
        for norm_feature in norm_features:
            df[norm_feature] = kdb.transform(df[norm_feature].values.reshape(-1, 1))
    print(df[['DRtg_0','DRtg_35']])
    print(df[['+/-_0','+/-_35']])
    print(df.info())
    print(df.columns)
    return df

def data_prerpocess(df, training = True):
    
    #V_0：全有
    used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
        'PTS', '+/-','TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']
    #v_1去除+-，DRtg
    used_features = ['MP','FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
        'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'BPM']
    #V_2去除MP,+-，DRtg
    # used_features = ['FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg','BPM']
    #v_3 +-,'BPM'
    # used_features = ['MP','FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg','DRtg']
    # V_4去除PTS,+-,'ORtg','BPM'
    # used_features = ['MP','FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #      'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'DRtg']
    #v_5 去除+-,'ORtg','DRtg', 'BPM'
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%']
    #v_6 去除+-,'ORtg','DRtg', 'BPM','PTS'
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #      'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%']
    #v_7 缩小+-,'BPM','ORtg','DRtg', 
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', '+/-','TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']
    #v_8 去除+-,'BPM','ORtg',通过模糊化降低'DRtg'影响值
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'DRtg']
    #v_9 去除+-,'BPM',通过模糊化降低'DRtg','ORtg'影响值
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'DRtg','ORtg']
    #v_10 去除+-,通过模糊化降低'DRtg','ORtg'影响值
    used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
        'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%','ORtg', 'DRtg', 'BPM']
    #v_11 模糊化+-,'DRtg'
    used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
        'PTS', '+/-','TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']
    X_list = []
    Y_list = []
    total_over_features = []
    for i in range(36):
        for feature in used_features:
            feature = feature + '_' + str(i)
            total_over_features.append(feature)
    
    print('选择的特征数量：',len(total_over_features))
    X = df[total_over_features].astype(float)
    Y = df['result'].astype(int)
    X_list.append(X)
    Y_list.append(Y)
    # print('处理后数据：',len(X))
 
    # #打乱同一个队球员顺序添加训练数据进去
    # random.seed(0)
    # for sample in range(15):
    #     total_over_features = []
    #     for i in random.sample(range(18), 18):  # 打乱顺序
    #         for feature in used_features:
    #             new_feature = feature + '_' + str(i)
    #             total_over_features.append(new_feature)
                
    #     for i in random.sample(range(18,36), 18):  # 打乱顺序
    #         for feature in used_features:
    #             new_feature = feature + '_' + str(i)
    #             total_over_features.append(new_feature)
                
    #     print('特征顺序：',total_over_features)
    #     X1 = df[total_over_features].astype(float)
    #     Y1 = df['result'].astype(int)
    #     X_list.append(X1)
    #     Y_list.append(Y1)
        
    X = pd.concat(X_list, ignore_index=True)
    Y = pd.concat(Y_list, ignore_index=True)    
    print('处理后数据：',len(X))
    
    if training:
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42) 
        return X_train[total_over_features], X_test[total_over_features], Y_train, Y_test
    else:
        return X[total_over_features], Y, total_over_features

def lgb_train(X_train, X_test, y_train, y_test):
    norm_features = []
    if  DRtg_Fuzzification:
        for i in range(36):
            for feat in ['DRtg']:
                feature = feat + '_' + str(i)
                norm_features.append(feature)
    if  PM_Fuzzification:
        for i in range(36):
            for feat in ['+/-']:
                feature = feat + '_' + str(i)
                norm_features.append(feature)
    categorical_feature = norm_features
    print(categorical_feature)
    model = lightgbm.LGBMClassifier(objective='binary', learning_rate=0.1, num_leaves = 30, min_data_in_leaf = 100, feature_fraction = 0.9, max_depth = 8, n_estimators=1500)
    callbacks = [log_evaluation(period=1), early_stopping(stopping_rounds=10)]
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='binary_logloss', callbacks = callbacks, categorical_feature = categorical_feature)
    joblib.dump(model, 'model/nba_mvp_lgb_v12.pkl')
    model = joblib.load('model/nba_mvp_lgb_v12.pkl')
    y_pred = model.predict(X_test, num_iteration=model.best_iteration_)


    print(np.average(y_pred))
    print('The acc of prediction is {}'.format(accuracy_score(y_test, y_pred)))
    print('The precision of prediction is {}'.format(precision_score(y_test, y_pred)))
    print('The recall of prediction is {}'.format(recall_score(y_test, y_pred)))

    X = pd.concat((X_train, X_test))
    y = np.append(y_train, y_test)
    y_all_pred = model.predict(X, num_iteration=model.best_iteration_)
    print('The acc of all prediction is {}'.format(accuracy_score(y, y_all_pred)))
    print('The precision of all prediction is {}'.format(precision_score(y, y_all_pred)))
    print('The recall of all prediction is {}'.format(recall_score(y, y_all_pred)))

def rawdata_load(extract_folder):
    file_path_list = []
    df_list = []
    # 遍历解压后的文件夹，并读取每个 CSV 文件
    for root, dirs, files in os.walk(extract_folder):
        for file in files:
            if file.endswith("basic_stats.csv"):
                file_path = os.path.join(root, file)
                file_path_list.append(file_path)
                # print(file_path)
                # 使用 pandas 读取 CSV 文件
                df_basic = pd.read_csv(file_path).drop_duplicates()
                df_advanced = pd.read_csv(file_path.replace('basic_stats.csv','advanced_stats.csv')).drop_duplicates()
                if df_basic['PTS'].dtype=='object':
                    df_basic = df_basic.drop(df_basic.index[-1])
                    df_advanced = df_advanced.drop(df_advanced.index[-1])
                # 连接两个文件：63162
                df = pd.concat([df_basic, df_advanced], axis=1)
                df = df.loc[:,~df.columns.duplicated()]
                # 将文件名作为新列添加到 DataFrame
                df['file_name'] = file_path.split('/')[-1]
                df_list.append(df)
    print(len(file_path_list),len(df_list)) 
    max_player = 18

    features = ['starters', 'team', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
        'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
        'PTS', '+/-', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
        'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM',
        'file_name']
    # 连接所有球员数据
    new_features = []
    for i in range(max_player*2):
        for feat in features:
            new_feat = feat + '_' + str(i)
            new_features.append(new_feat)
    # print(len(features), len(new_features))

    result_features = ['score_ourside', 'score_opposite', 'result']

    new_data = pd.DataFrame(columns = new_features + result_features)
    for i in tqdm(range(len(file_path_list))):
        # for i in range(10):
        path = file_path_list[i]
        # print(path.split('/')[-1])
        df1 = df_list[i]
        filled_df1 = df1.reindex(range(max_player))
        #对手球队数据
        a = path.split('/')[-1].split('_')[3]
        b = path.split('/')[-1].split('_')[5]
        oppo_path = path.replace(a+'_vs_'+b, b+'_vs_'+a)
        # print(oppo_path,file_path_list.index(oppo_path))
        df2 = df_list[file_path_list.index(oppo_path)]
        filled_df2 = df2.reindex(range(max_player))
        # print(filled_df1["PTS"],filled_df2["PTS"])
        score_ourside = filled_df1["PTS"].astype(float).sum()
        score_opposite = filled_df2["PTS"].astype(float).sum()
        if score_ourside > score_opposite:
            result = 1
        else:
            result = 0
        
        feature_data1 = []
        feature_data2 = []
        for num in range(max_player):
            feature_data1 += filled_df1.loc[num, features].tolist() 
            feature_data2 += filled_df2.loc[num, features].tolist() 
            
        new_data.loc[i, new_features + result_features] = feature_data1 + feature_data2 + [score_ourside, score_opposite, result]
    return new_data

if __name__ == "__main__":
    data_path = './data/nba_mvp.csv'
    df1 = pd.read_csv(data_path)
    #加上24年数据
    folder_path = 'data/basketball_data_info/'
    df2 = rawdata_load(folder_path)
    df = pd.concat([df1, df2], axis=0)  
    print(len(df1),len(df2),len(df))
    data = data_load(df)
    X_train, X_test, y_train, y_test = data_prerpocess(data)
    lgb_train(X_train, X_test, y_train, y_test)

/tmp/ipykernel_2331208/2835855775.py:310: DtypeWarning: Columns (37,74,111,148,185,222,259,296,333,370,407,444,481,495,496,497,532,533,534,535,570,571,572,573,608,609,610,611,646,647,648,649,684,721,758,795,832,869,906,943,980,1017,1054,1091,1128,1165,1179,1180,1181,1216,1217,1218,1219,1254,1255,1256,1257,1292,1293,1294,1295,1330,1331,1332,1333,1368) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(data_path)


92 92


100%|██████████| 92/92 [00:00<00:00, 99.56it/s] 


63162 92 63254
清洗后数据： 63204


/tmp/ipykernel_2331208/2835855775.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[time_features] = df[time_features].fillna('0:0')
/tmp/ipykernel_2331208/2835855775.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[time_features] = df[time_features].map(lambda x: '0:0' if ':' not in str(x) else x)
/tmp/ipykernel_2331208/2835855775.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

(2275344, 1)


/root/miniconda3/envs/mvp_shap/lib/python3.9/site-packages/sklearn/preprocessing/_discretization.py:322: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/tmp/ipykernel_2331208/2835855775.py:81: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[norm_features] = df[norm_features].applymap(lambda x: 0 if x < 0 else x)


(2275344, 1)
    DRtg_0  DRtg_35
0      3.0      0.0
1      4.0      0.0
2      1.0      0.0
3      3.0      0.0
4      4.0      0.0
..     ...      ...
87     4.0      0.0
88     4.0      0.0
89     4.0      0.0
90     2.0      0.0
91     1.0      0.0

[63204 rows x 2 columns]
    +/-_0  +/-_35
0     0.0     0.0
1     0.0     0.0
2     0.0     0.0
3     0.0     0.0
4     0.0     0.0
..    ...     ...
87    0.0     0.0
88    0.0     0.0
89    0.0     0.0
90    0.0     0.0
91    0.0     0.0

[63204 rows x 2 columns]


/root/miniconda3/envs/mvp_shap/lib/python3.9/site-packages/sklearn/preprocessing/_discretization.py:322: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(


<class 'pandas.core.frame.DataFrame'>
Index: 63204 entries, 0 to 91
Columns: 1372 entries, Unnamed: 0 to result
dtypes: float64(1047), int64(193), object(132)
memory usage: 662.1+ MB
None
Index(['Unnamed: 0', 'starters_0', 'team_0', 'MP_0', 'FG_0', 'FGA_0', 'FG%_0',
       '3P_0', '3PA_0', '3P%_0',
       ...
       'BLK%_35', 'TOV%_35', 'USG%_35', 'ORtg_35', 'DRtg_35', 'BPM_35',
       'file_name_35', 'score_ourside', 'score_opposite', 'result'],
      dtype='object', length=1372)
选择的特征数量： 1260
处理后数据： 63204
['DRtg_0', 'DRtg_1', 'DRtg_2', 'DRtg_3', 'DRtg_4', 'DRtg_5', 'DRtg_6', 'DRtg_7', 'DRtg_8', 'DRtg_9', 'DRtg_10', 'DRtg_11', 'DRtg_12', 'DRtg_13', 'DRtg_14', 'DRtg_15', 'DRtg_16', 'DRtg_17', 'DRtg_18', 'DRtg_19', 'DRtg_20', 'DRtg_21', 'DRtg_22', 'DRtg_23', 'DRtg_24', 'DRtg_25', 'DRtg_26', 'DRtg_27', 'DRtg_28', 'DRtg_29', 'DRtg_30', 'DRtg_31', 'DRtg_32', 'DRtg_33', 'DRtg_34', 'DRtg_35', '+/-_0', '+/-_1', '+/-_2', '+/-_3', '+/-_4', '+/-_5', '+/-_6', '+/-_7', '+/-_8', '+/-_9', '+/-_10',

In [617]:
year = 2022

In [618]:
# 24,22
season = 'Finals/'
model_path = 'model/nba_mvp_lgb_v12.pkl'
if year == 2024:
    # folder_path = 'data/basketball_data_info/'
    day1 ='2024_06_06'
    day2 ='2024_06_09'
    day3 ='2024_06_12'
    day4 ='2024_06_14'
    day5 ='2024_06_17'
    # folder_list = [folder_path+day1,folder_path+day2,folder_path+day3, folder_path+day4, folder_path+day5]
    final_data = data[data.apply(lambda x: x.astype(str).str.contains('2024_06_06|2024_06_09|2024_06_12|2024_06_14|2024_06_17'
                            , regex=True)).any(axis=1)]
if year == 2023:
    folder_path = 'data/nba/basketball_reference_part2_finish/'
    day1 ='2023_06_01'
    day2 ='2023_06_04'
    day3 ='2023_06_07'
    day4 ='2023_06_09'
    day5 ='2023_06_12'
    folder_list = [folder_path+day1,folder_path+day2,folder_path+day3, folder_path+day4, folder_path+day5]
    final_data = data[data.apply(lambda x: x.astype(str).str.contains('2023_06_01|2023_06_04|2023_06_07|2023_06_09|2023_06_12'
                            , regex=True)).any(axis=1)]
if year == 2022:
    folder_path = 'data/nba/basketball_reference_part2_finish/'
    day1 ='2022_06_02'
    day2 ='2022_06_05'
    day3 ='2022_06_08'
    day4 ='2022_06_10'
    day5 ='2022_06_13'
    day6 ='2022_06_16'
    folder_list = [folder_path+day1,folder_path+day2,folder_path+day3, folder_path+day4, folder_path+day5, folder_path+day6]
    final_data = data[data.apply(lambda x: x.astype(str).str.contains('2022_06_02|2022_06_05|2022_06_08|2022_06_10|2022_06_13|2022_06_16'
                            , regex=True)).any(axis=1)]
if year == 2021:
    folder_path = 'data/nba/basketball_reference_part2_finish/'
    day1 ='2021_07_06'
    day2 ='2021_07_08'
    day3 ='2021_07_11'
    day4 ='2021_07_14'
    day5 ='2021_07_17'
    day6 ='2021_07_20'
    folder_list = [folder_path+day1,folder_path+day2,folder_path+day3, folder_path+day4, folder_path+day5, folder_path+day6]
    final_data = data[data.apply(lambda x: x.astype(str).str.contains('2021_07_06|2021_07_08|2021_07_11|2021_07_14|2021_07_17|2021_07_20'
                            , regex=True)).any(axis=1)]
if year == 2020:
    folder_path = 'data/nba/basketball_reference_part2_finish/'
    day1 ='2020_09_30'
    day2 ='2020_10_02'
    day3 ='2020_10_04'
    day4 ='2020_10_06'
    day5 ='2020_10_09'
    day6 ='2020_10_11'
    folder_list = [folder_path+day1,folder_path+day2,folder_path+day3, folder_path+day4, folder_path+day5,folder_path+day6]
    final_data = data[data.apply(lambda x: x.astype(str).str.contains('2020_09_30|2020_10_02|2020_10_04|2020_10_06|2020_10_09|2020_10_11'
                            , regex=True)).any(axis=1)]
if year == 2019:
    folder_path = 'data/nba/basketball_reference_part2_finish/'
    day1 ='2019_05_30'
    day2 ='2019_06_02'
    day3 ='2019_06_05'
    day4 ='2019_06_07'
    day5 ='2019_06_10'
    day6 ='2019_06_13'
    folder_list = [folder_path+day1,folder_path+day2,folder_path+day3, folder_path+day4, folder_path+day5,folder_path+day6]
    final_data = data[data.apply(lambda x: x.astype(str).str.contains('2019_05_30|2019_06_02|2019_06_05|2019_06_07|2019_06_10|2019_06_13'
                            , regex=True)).any(axis=1)]
if year == 2018:
    folder_path = 'data/nba/basketball_reference_part2_finish/'
    day1 ='2018_05_31'
    day2 ='2018_06_03'
    day3 ='2018_06_06'
    day4 ='2018_06_08'
    folder_list = [folder_path+day1,folder_path+day2,folder_path+day3, folder_path+day4]
    final_data = data[data.apply(lambda x: x.astype(str).str.contains('2018_05_31|2018_06_03|2018_06_06|2018_06_08'
                            , regex=True)).any(axis=1)]
if year == 2017:
    folder_path = 'data/nba/basketball_reference_part2_finish/'
    day1 ='2017_06_01'
    day2 ='2017_06_04'
    day3 ='2017_06_07'
    day4 ='2017_06_09'
    day5 ='2017_06_12'
    folder_list = [folder_path+day1,folder_path+day2,folder_path+day3, folder_path+day4, folder_path+day5]
    final_data = data[data.apply(lambda x: x.astype(str).str.contains('2017_06_01|2017_06_04|2017_06_07|2017_06_09|2017_06_12'
                            , regex=True)).any(axis=1)]
if year == 2016:
    folder_path = 'data/nba/basketball_reference_part2_finish/'
    day1 ='2016_06_02'
    day2 ='2016_06_05'
    day3 ='2016_06_08'
    day4 ='2016_06_10'
    day5 ='2016_06_13'
    day6 ='2016_06_16'
    day7 ='2016_06_19'
    folder_list = [folder_path+day1,folder_path+day2,folder_path+day3, folder_path+day4, folder_path+day5, folder_path+day6, folder_path+day7]
    final_data = data[data.apply(lambda x: x.astype(str).str.contains('2016_06_02|2016_06_05|2016_06_08|2016_06_10|2016_06_13|2016_06_16|2016_06_19'
                            , regex=True)).any(axis=1)]
if year == 2015:
    folder_path = 'data/nba/basketball_reference_part2_finish/'
    day1 ='2015_06_04'
    day2 ='2015_06_07'
    day3 ='2015_06_09'
    day4 ='2015_06_11'
    day5 ='2015_06_14'
    day6 ='2015_06_16'
    folder_list = [folder_path+day1,folder_path+day2,folder_path+day3, folder_path+day4, folder_path+day5, folder_path+day6]
    final_data = data[data.apply(lambda x: x.astype(str).str.contains('2015_06_04|2015_06_07|2015_06_09|2015_06_11|2015_06_14|2015_06_16'
                            , regex=True)).any(axis=1)]
if year == 2014:
    folder_path = 'data/nba/basketball_reference_part2_finish/'
    day1 ='2014_06_05'
    day2 ='2014_06_08'
    day3 ='2014_06_10'
    day4 ='2014_06_12'
    day5 ='2014_06_15'
    folder_list = [folder_path+day1,folder_path+day2,folder_path+day3, folder_path+day4, folder_path+day5]
    final_data = data[data.apply(lambda x: x.astype(str).str.contains('2014_06_05|2014_06_08|2014_06_10|2014_06_12|2014_06_15'
                            , regex=True)).any(axis=1)]
if year == 2013:
    folder_path = 'data/nba/basketball_reference/'
    day1 ='2013_06_06'
    day2 ='2013_06_09'
    day3 ='2013_06_11'
    day4 ='2013_06_13'
    day5 ='2013_06_16'
    day6 ='2013_06_18'
    day7 ='2013_06_20'
    folder_list = [folder_path+day1,folder_path+day2,folder_path+day3, folder_path+day4, folder_path+day5, folder_path+day6, folder_path+day7]
    final_data = data[data.apply(lambda x: x.astype(str).str.contains('2013_06_06|2013_06_09|2013_06_11|2013_06_13|2013_06_16|2013_06_18|2013_06_20'
                            , regex=True)).any(axis=1)]
if year == 2012:
    folder_path = 'data/nba/basketball_reference/'
    day1 ='2012_06_12'
    day2 ='2012_06_14'
    day3 ='2012_06_17'
    day4 ='2012_06_19'
    day5 ='2012_06_21'
    folder_list = [folder_path+day1,folder_path+day2,folder_path+day3, folder_path+day4, folder_path+day5]
    final_data = data[data.apply(lambda x: x.astype(str).str.contains('2012_06_12|2012_06_14|2012_06_17|2012_06_19|2012_06_21'
                            , regex=True)).any(axis=1)]
if year == 2011:
    folder_path = 'data/nba/basketball_reference/'
    day1 ='2011_05_31'
    day2 ='2011_06_02'
    day3 ='2011_06_05'
    day4 ='2011_06_07'
    day5 ='2011_06_09'
    day6 ='2011_06_12'
    folder_list = [folder_path+day1,folder_path+day2,folder_path+day3, folder_path+day4, folder_path+day5, folder_path+day6]
    final_data = data[data.apply(lambda x: x.astype(str).str.contains('2011_05_31|2011_06_02|2011_06_05|2011_06_07|2011_06_09|2011_06_12'
                            , regex=True)).any(axis=1)]
if year == 2010:
    folder_path = 'data/nba/basketball_reference/'
    day1 ='2010_06_03'
    day2 ='2010_06_06'
    day3 ='2010_06_08'
    day4 ='2010_06_10'
    day5 ='2010_06_13'
    day6 ='2010_06_15'
    day7 ='2010_06_17'
    folder_list = [folder_path+day1,folder_path+day2,folder_path+day3, folder_path+day4, folder_path+day5, folder_path+day6, folder_path+day7]
    final_data = data[data.apply(lambda x: x.astype(str).str.contains('2010_06_03|2010_06_06|2010_06_08|2010_06_10|2010_06_13|2010_06_15|2010_06_17'
                            , regex=True)).any(axis=1)]
if year == 2009:
    folder_path = 'data/nba/basketball_reference/'
    day1 ='2009_06_04'
    day2 ='2009_06_07'
    day3 ='2009_06_09'
    day4 ='2009_06_11'
    day5 ='2009_06_14'
    folder_list = [folder_path+day1,folder_path+day2,folder_path+day3, folder_path+day4, folder_path+day5]
    final_data = data[data.apply(lambda x: x.astype(str).str.contains('2009_06_04|2009_06_07|2009_06_09|2009_06_11|2009_06_14'
                            , regex=True)).any(axis=1)]
if year == 2008:
    folder_path = 'data/nba/basketball_reference/'
    day1 ='2008_06_05'
    day2 ='2008_06_08'
    day3 ='2008_06_10'
    day4 ='2008_06_12'
    day5 ='2008_06_15'
    day6 ='2008_06_17'
    folder_list = [folder_path+day1,folder_path+day2,folder_path+day3, folder_path+day4, folder_path+day5, folder_path+day6]
    final_data = data[data.apply(lambda x: x.astype(str).str.contains('2008_06_05|2008_06_08|2008_06_10|2008_06_12|2008_06_15|2008_06_17'
                            , regex=True)).any(axis=1)]
if year == 2007:
    folder_path = 'data/nba/basketball_reference/'
    day1 ='2007_06_07'
    day2 ='2007_06_10'
    day3 ='2007_06_12'
    day4 ='2007_06_14'
    folder_list = [folder_path+day1,folder_path+day2,folder_path+day3, folder_path+day4]
    final_data = data[data.apply(lambda x: x.astype(str).str.contains('2007_06_07|2007_06_10|2007_06_12|2007_06_14'
                            , regex=True)).any(axis=1)]
if year == 2006:
    folder_path = 'data/nba/basketball_reference/'
    day1 ='2006_06_08'
    day2 ='2006_06_11'
    day3 ='2006_06_13'
    day4 ='2006_06_15'
    day5 ='2006_06_18'
    day6 ='2006_06_20'
    folder_list = [folder_path+day1,folder_path+day2,folder_path+day3, folder_path+day4, folder_path+day5, folder_path+day6]
    final_data = data[data.apply(lambda x: x.astype(str).str.contains('2006_06_08|2006_06_11|2006_06_13|2006_06_15|2006_06_18|2006_06_20'
                            , regex=True)).any(axis=1)]


def compute_shap_values(shap_values_path, X):
    shap_values = np.load(shap_values_path)
    feat_num = len(shap_values[0]) // 36 #每个球员有33个特征
    mvp_values = []
    X = X.fillna(0)
    for i in tqdm(range(len(X))):
        shap_value = shap_values[i]
        mvp_value = 0
        for j in range(len(shap_value)):
            mvp_value += shap_value[j]#*X.loc[i,total_over_features[j]]
            if j % feat_num == feat_num - 1:
                mvp_values.append(mvp_value)
                mvp_value = 0
    abs_array = np.abs(np.array(mvp_values).reshape(-1,36))
    # print('1111111111111:',mvp_values,X)
    sum_first_three = np.sum(abs_array[:, :18], axis=1, keepdims=True)
    x = 0.5
    normalized_first_three = (abs_array[:, :18] / sum_first_three) * x

    # 后三列数值除以后三列的和并乘以系数（1-x）
    sum_last_three = np.sum(abs_array[:, 18:], axis=1, keepdims=True)
    normalized_last_three = (abs_array[:, 18:] / sum_last_three) * (1 - x)

    # 合并处理后的数组
    result_array = np.hstack((normalized_first_three, normalized_last_three))
    return result_array, np.array(mvp_values).reshape(-1,36), shap_values

def explain(X, model_path, shap_file_name):

    model = joblib.load(model_path)
    explainer = shap.TreeExplainer(model)
    # print(explainer.expected_value)
    shap_values = explainer.shap_values(X)
    
    np.save(shap_file_name, shap_values)

    shap_values = np.load(shap_file_name)
    print(shap_values.shape)
   
# 队内排名
def rank(values, order = 1):
    if order == 1:
        sorted_indices = np.argsort(values)[::-1]
    else:
        sorted_indices = np.argsort(values)
    ranks = np.empty_like(sorted_indices)
    ranks[sorted_indices] = np.arange(1, len(sorted_indices) + 1)

    return ranks, sorted_indices[0]

def process_data(df, feat_values, shap_values):
    df, feat_values, shap_values
    #V_0
    used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
        'PTS', '+/-','TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']
    # #v_1去除+-，DRtg
    used_features = ['MP','FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
        'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'BPM']
    #V_2
    # used_features = ['FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg','BPM']
    #v_3
    # used_features = ['MP','FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg','DRtg']
    # V_4
    # used_features = ['MP','FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #      'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'DRtg']
    #v_5
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%']
    # v_6 去除+-,'ORtg','DRtg', 'BPM','PTS'
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #      'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%']
    #v_7 缩小+-,'BPM','ORtg','DRtg', 
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', '+/-','TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']
    #v_8 去除+-,'BPM','ORtg',通过模糊化降低'DRtg'影响值
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'DRtg']
    #v_9 去除+-,'BPM',通过模糊化降低'DRtg','ORtg'影响值
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'DRtg','ORtg']
    #v_10 去除+-,通过模糊化降低'DRtg','ORtg'影响值
    used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
        'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%','ORtg', 'DRtg', 'BPM']
    #v_11 去除+-,'DRtg',打乱球员位置
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%','ORtg', 'BPM']
    #v_12 模糊化+-,'DRtg'
    used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
        'PTS', '+/-','TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']
    
    feature_value = []
    id_feature = 'starters'
    id_features = [id_feature + '_' + str(i) for i in range(36)]
    

    score_features = ['score_ourside', 'score_opposite']
    total_over_features = []
    feat_num = len(used_features)
    
    for feature in used_features:
        feature_value.append(feature)
        feature_value.append(feature + '_shap_value')
    value_name = [feature_value[i] for i in range(1, len(feature_value), 2)]

    for i in range(36):
        for feature in used_features:
            feature = feature + '_' + str(i)
            total_over_features.append(feature)
  
    #ds, role_id
    new_data = pd.DataFrame(columns = ['file_name', 'result', 'starters', 'home_team', 'team_score'] + feature_value + [ 'shap_score', 'shap_order_team'])
    for i in tqdm(range(len(df))):
        result = df.loc[i, 'result']
        match_id = df.loc[i, 'file_name_0']
      
        self_shap_values = shap_values[i][:18]
        opp_shap_values = shap_values[i][18:]
        
        for j in range(18):
            #ds, role_id, feat_shap_value
            new_data.loc[36 * i + j, ['file_name', 'result', 'starters', 'team_score'] + used_features] = df.loc[i, ['file_name_0', 'result',  id_features[j], score_features[0]] + total_over_features[j * feat_num : (j + 1) * feat_num]].tolist()
            new_data.loc[36 * i + j, 'home_team'] = '1'
            new_data.loc[36 * i + j, value_name] = feat_values[i][j * feat_num : (j + 1) * feat_num] #n * feat_num * 6
            
        rows = [36 * i + n for n in range(18)]
        shap_ranks, shap_index1 = rank(self_shap_values)
        new_data.loc[rows, 'shap_order_team'] = shap_ranks
        
        
        for j in range(18, 36):
            new_data.loc[36 * i + j, ['file_name',  'starters', 'team_score'] + used_features] = df.loc[i, ['file_name_18', id_features[j], score_features[1]] + total_over_features[j * feat_num : (j + 1) * feat_num]].tolist()
            new_data.loc[36 * i + j, 'home_team'] = '0'
            new_data.loc[36 * i + j, 'result'] = 1 - df.loc[i, 'result']
            new_data.loc[36 * i + j, value_name] = feat_values[i][j * feat_num : (j + 1) * feat_num]
           
        rows = [36 * i + n for n in range(18,36)]
        shap_ranks, shap_index2 = rank(opp_shap_values, -1)
        new_data.loc[rows, 'shap_order_team'] = shap_ranks

    new_data['shap_score'] = shap_values.reshape(-1)
    return new_data

print('{}Finals比赛数据：'.format(year),len(final_data))
# print(data['file_name_0'])
X, Y, total_over_features = data_prerpocess(final_data, training= False) 
shap_file_name = './shap/'+season+'{}_nba_mvp_shap_values_v12.npy'.format(year)
# if not os.path.exists(shap_file_name):
explain(X, model_path, shap_file_name)
money_values, shap_values, feat_values = compute_shap_values(shap_file_name, X)


print(feat_values.shape,shap_values.shape)
processed_data = process_data(final_data.reset_index(drop = True), feat_values, shap_values)
processed_data.to_csv('./data/'+season+'{}_nba_mvp_values_v12.csv'.format(year))
# 绘制summary分析图并保存
def summary_bar_plot(shap_values, max_display=20, show=False, save_path=None):
    plt.style.use('ggplot')
    shap.plots.bar(shap_values, max_display=max_display, show=show)
    if save_path:
        plt.savefig(save_path.encode(encoding='utf-8'), dpi=200, bbox_inches='tight')
    plt.close('all')

def summary_dot_plot(shap_values, max_display=20, show=False, save_path=None):
    plt.style.use('ggplot')
    shap.plots.beeswarm(shap_values, max_display=max_display, show=show)
    if save_path:
        plt.savefig(save_path.encode(encoding='utf-8'), dpi=200, bbox_inches='tight')
    plt.close('all')

# 绘制summary分析图并保存
save_path = 'shap/'+season+'{}_nba_mvp_summary_dot_v12.png'.format(year)
summary_dot_plot(shap.Explanation(
values=feat_values, 
# base_values = 0,
data=np.array(X),
feature_names=total_over_features), max_display=50, show=False, save_path=save_path)

save_path = 'shap/'+season+'{}_nba_mvp_summary_bar_v12.png'.format(year)
summary_bar_plot(shap.Explanation(
values=feat_values, 
# base_values = np.array(0),
data=np.array(X),
feature_names=total_over_features), max_display=50, show=False, save_path=save_path)





2022Finals比赛数据： 12
选择的特征数量： 1260
处理后数据： 12


/root/miniconda3/envs/mvp_shap/lib/python3.9/site-packages/shap/explainers/_tree.py:448: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn('LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray')


(12, 1260)


100%|██████████| 12/12 [00:00<00:00, 5596.76it/s]


(12, 1260) (12, 36)


100%|██████████| 12/12 [00:00<00:00, 15.63it/s]


In [619]:
# model_path = 'model/nba_mvp_lgb_v12.pkl'
model = joblib.load(model_path)
print(Y)
y = model.predict(X, num_iteration=model.best_iteration_)
# y = model.predict_proba(X, num_iteration=model.best_iteration_)
print(y)

0     0
1     1
2     1
3     0
4     0
5     1
6     0
7     1
8     1
9     0
10    1
11    0
Name: result, dtype: int64
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[0 1 1 0 0 1 0 1 1 0 1 0]


In [620]:
#球员在所有胜场的shapley值排名平均
data_value = pd.read_csv('./data/'+season+'{}_nba_mvp_values_v12.csv'.format(year))
data_value= data_value[data_value['MP'] != 0.0]
# print('上场人数：',len(data_value))
result_df = \
data_value[data_value['starters'].
           map(data_value['starters'].value_counts()) > len(final_data)-1]
#取出胜方球员
df = result_df
result_1 = df[df['result'] == 1]
max_result_1_count = result_1['starters'].value_counts().max()
max_result_1_starters = result_1['starters'].value_counts()[result_1['starters'].value_counts() == max_result_1_count].index
print(max_result_1_starters)
result_df = df[df['starters'].isin(max_result_1_starters)]
#胜场
result_df_win = result_df[result_df['result'] == 1]
mvp_by_shap = result_df_win.groupby('starters')['shap_order_team'].mean().sort_values()
print(len(mvp_by_shap),mvp_by_shap[:10])

Index(['Andrew Wiggins', 'Draymond Green', 'Klay Thompson', 'Stephen Curry',
       'Otto Porter Jr.', 'Kevon Looney', 'Jordan Poole'],
      dtype='object', name='starters')
7 starters
Draymond Green     3.000
Stephen Curry      3.375
Andrew Wiggins     4.375
Kevon Looney       4.750
Klay Thompson      5.500
Jordan Poole       6.125
Otto Porter Jr.    9.000
Name: shap_order_team, dtype: float64


In [621]:
#球员在所有场的shapley值排名平均
mvp_by_shap = result_df.groupby('starters')['shap_order_team'].mean().sort_values()
print(len(mvp_by_shap),mvp_by_shap[:10])

7 starters
Stephen Curry      6.750000
Draymond Green     7.833333
Kevon Looney       7.833333
Andrew Wiggins     8.333333
Klay Thompson      8.916667
Jordan Poole       9.416667
Otto Porter Jr.    9.583333
Name: shap_order_team, dtype: float64


In [622]:
#球员在所有场的shapley值平均
import pandas as pd
from scipy.stats import spearmanr
# result_df = pd.read_csv('./data/'+season+'{}_nba_mvp_values_v12.csv'.format(year))
result_df1 = result_df[result_df['home_team'] == 1]
result_df2 = result_df[result_df['home_team'] == 0]

# print(result_df['starters'].value_counts())
mvp_by_shap1 = result_df1.groupby('starters')['shap_score'].mean()
mvp_by_shap2 = result_df2.groupby('starters')['shap_score'].mean()

mvp_by_shap = (mvp_by_shap1 - mvp_by_shap2).sort_values(ascending=False)
print(mvp_by_shap[:10])

starters
Stephen Curry      0.915368
Kevon Looney       0.408587
Otto Porter Jr.    0.286604
Andrew Wiggins     0.271679
Draymond Green     0.243139
Klay Thompson     -0.004156
Jordan Poole      -0.159121
Name: shap_score, dtype: float64


In [623]:
#输球负贡献置零
# result_df = pd.read_csv('./data/'+season+'{}_nba_mvp_values_v12.csv'.format(year))
result_df1 = result_df[result_df['home_team'] == 1]
result_df2 = result_df[result_df['home_team'] == 0]
#负场
result_df_loss1 = result_df1[result_df1['result'] == 0]
result_df_loss1['shap_score'][result_df_loss1['shap_score'] < 0] = 0
result_df_loss2 = result_df2[result_df2['result'] == 0]
result_df_loss2['shap_score'][result_df_loss2['shap_score'] > 0] = 0
result_df1 = pd.concat([result_df_loss1, result_df1[result_df1['result'] == 1]])
result_df2 = pd.concat([result_df_loss2, result_df2[result_df2['result'] == 1]])
# print(result_df['starters'].value_counts())
mvp_by_shap1 = result_df1.groupby('starters')['shap_score'].mean()
mvp_by_shap2 = result_df2.groupby('starters')['shap_score'].mean()

mvp_by_shap = (mvp_by_shap1 - mvp_by_shap2).sort_values(ascending=False)
print(mvp_by_shap[:10])

starters
Stephen Curry      1.648865
Draymond Green     1.568513
Andrew Wiggins     1.338613
Kevon Looney       1.202349
Klay Thompson      1.044415
Jordan Poole       0.861054
Otto Porter Jr.    0.589318
Name: shap_score, dtype: float64


/tmp/ipykernel_2331208/2411647944.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  result_df_loss1['shap_score'][result_df_loss1['shap_score'] < 0] = 0
/tmp/ipykernel_2331208/2411647944.py:7: SettingWithCopyWarning: 
A value is trying to 